In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
test = pd.read_csv("../input/cat-in-the-dat/test.csv")
train = pd.read_csv("../input/cat-in-the-dat/train.csv")

In [ ]:
train.head(10)

In [ ]:
test.head(10)

Check for missing values 

In [ ]:
train.isna().sum(axis=0)


In [ ]:
test.isna().sum(axis=0)

## One Hot Encoding

In [ ]:
num_cols = 0

for c in train.columns[1:-1]:
    num_cols += len(np.unique(train[c]))
    
print(num_cols)

In [ ]:
train.nunique()

In [ ]:
features = train.columns.to_list()
features.remove('id')
features.remove('target')
print(features)

In [ ]:
encoder = OneHotEncoder(sparse = True, handle_unknown='ignore')
encoder.fit(train[features])
X_train = encoder.transform(train[features])
X_test = encoder.transform(test[features])

y_train = train.target.values

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape: ', X_test.shape)

In [ ]:
pd.DataFrame(X_test)

# Logistic Regression

In [ ]:
lr_mod = LogisticRegression(solver='liblinear')
lr_mod.fit(X_train, y_train)
print(lr_mod.score(X_train, y_train))

In [ ]:
lr_cv_results = cross_val_score(lr_mod, X_train, y_train, cv=5, scoring='roc_auc')

print('Validation AUC by fold: ', lr_cv_results)
print('Average Validation AUC: ', np.mean(lr_cv_results))

# Logistic Regression: Hyper-Parameter Tuning

In [ ]:
%%time 

lr_param_grid = {
    'C': [0.01, 0.1, 1]
}

lr_grid_search = GridSearchCV(lr_mod, lr_param_grid, cv=5, refit='True',n_jobs=-1)
lr_grid_search.fit(X_train, y_train)

lr_gs_res = lr_grid_search.cv_results_

print(lr_gs_res.keys())

In [ ]:
best_lr_model = lr_grid_search.best_estimator_

from sklearn.model_selection import cross_validate

score=cross_validate(best_lr_model, X_train, y_train, cv=3, scoring="roc_auc")["test_score"].mean()
print(f"{score:.6f}")

# Make prediction 

In [ ]:
pred=best_lr_model.predict_proba(X_test)[:,1]
print(pred[:10])

In [ ]:
submission = pd.DataFrame({
    'id' : test.id,
    'target' : pred
})
submission.head()

In [ ]:
submission.to_csv('my_submission.csv', index=False)